In [3]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
  Using cached xgboost-1.5.0-py3-none-manylinux2014_x86_64.whl (173.5 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, roc_curve, plot_roc_curve
from sklearn.svm import SVC

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import os
from glob import glob
from IPython.display import Image
import random
%matplotlib inline

In [3]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'data')
CheXpert_train_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_hidden_features_all.npy'))
CheXpert_train_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_labels_all.npy'))
CheXpert_valid_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_hidden_features_all.npy'))
CheXpert_valid_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_labels_all.npy'))
extra_valid_age_sex_df = pd.read_csv(os.path.join(DATA_DIR,'extra_valid_age_sex.csv'))
extra_valid_hidden_features = np.load(os.path.join(DATA_DIR,'extra_valid_hidden_features.npy'))
extra_valid_labels = np.load(os.path.join(DATA_DIR,'extra_valid_labels.npy'))
extra_valid_images = glob(os.path.join(DATA_DIR, 'extraValid', '*'))

In [7]:
label_headers = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'P. Effusion']
CheXpert_train_labels_all_df = pd.DataFrame(CheXpert_train_labels_all, columns=label_headers)
CheXpert_valid_labels_all_df = pd.DataFrame(CheXpert_valid_labels_all, columns=label_headers)
extra_valid_labels_df = pd.DataFrame(extra_valid_labels, columns=label_headers)

In [8]:
# Label smoothing. Single example can have multiple labels
total = 0
for i, row in enumerate(CheXpert_train_labels_all):
    count = 0
    for j, col in enumerate(row):
        if CheXpert_train_labels_all[i,j] == 1:
            count += 1
    if count >= 2:
        total += 1
print(total / i)

0.26183346537578384


In [9]:
gbc = GradientBoostingClassifier(n_estimators=1, learning_rate=0.1, max_depth=1, random_state=0)
multi_target_gbc = MultiOutputClassifier(gbc, n_jobs=-1)

In [10]:
gbc_fitted = multi_target_gbc.fit(CheXpert_train_hidden_features_all, CheXpert_train_labels_all)

In [11]:
gbc_fitted.score(CheXpert_valid_hidden_features_all, CheXpert_valid_labels_all)

0.41452991452991456

In [12]:
y_pred = gbc_fitted.predict_proba(CheXpert_valid_hidden_features_all)

In [13]:
y_pred = np.transpose([pred[:, 1] for pred in y_pred])

In [14]:
auc = roc_auc_score(CheXpert_valid_labels_all, y_pred)

In [4]:
# SVM
clf = SVC()

In [5]:
multi_clf = MultiOutputClassifier(clf, n_jobs=-1)

In [ ]:
clf_fitted = multi_clf.fit(CheXpert_train_hidden_features_all, CheXpert_train_labels_all)

In [ ]:
y_pred = clf_fitted(CheXpert_valid_hidden_features_all)